In [81]:
import os
from groq import Groq
from dotenv import load_dotenv
load_dotenv()
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain_groq import ChatGroq
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
import streamlit as st

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [14]:
docs = PyPDFLoader("/home/skynet/Documents/Gen_ai/GeetaGpt/data/bhagvatgeeta.pdf").load()

In [4]:
# splitter = RecursiveCharacterTextSplitter(chunk_size=8100,chunk_overlap=2000)

In [5]:
# corpus = splitter.split_documents(data)

In [4]:
model = ChatGroq(model="gemma2-9b-it",max_tokens=4050)
llm = OllamaLLM(model="gemma2:9b")
embeddings = OllamaEmbeddings(model="gemma2:9b")

In [7]:
# db = FAISS.from_documents(corpus, embeddings)

In [9]:
# db.save_local("bhagvatgeeta_faiss")

In [6]:
db = FAISS.load_local("bhagvatgeeta_faiss",embeddings=embeddings,allow_dangerous_deserialization=True)

In [7]:
query = "What is the meaning of life?"
results = db.similarity_search(query, k=1)

In [15]:
retriever = db.as_retriever()
# prompt = hub.pull('rlm/rag-prompt')

In [16]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [84]:
prompt_template = """
Prompt:

    Task: Answer questions based on the Bhagavad Gita teachings.

    Instructions:

        Retrieve relevant information: Search the Bhagavad Gita for passages that are most relevant to the query.
        Summarize key points: Extract the main ideas and concepts from the retrieved passages.
        Generate a comprehensive response: Combine the summarized information with the query to create a coherent and informative answer.
        Mention references: Clearly indicate the chapter and verse numbers from which the information is extracted.
        Mention Sholkas: Clearly include relevant shlokas from the Gita to support your response.
        Provide accurate information: Ensure that the information provided is accurate and consistent with the teachings of the Gita.
        Translate: Provide translations of the shlokas in Hindi to help explain their meaning.


    Guidelines:

        Stay true to the text: Ensure that your responses are consistent with the teachings of the Bhagavad Gita.
        Provide context: If applicable, provide additional context from the Gita to support your answer.
        Be respectful: Maintain a respectful and reverent tone when answering questions about the Bhagavad Gita.
        Be concise: Provide clear and concise responses that address the query directly.
        Stay focused: Ensure that your response is directly related to the Bhagavad Gita and avoids unrelated and out of context topics.
        Answering Style: Start with "Jai Shri Krishna" and end with "Jai Shri Krishna" in your answer.
        Out of scope: Avoid providing personal opinions or interpretations that are not supported by the text of the Gita.
        Out of Context: Avoid providing information that is not directly related to the query or the teachings of the Gita.
        Inaccurate Information: Avoid providing information that is inaccurate or inconsistent with the teachings of the Gita.
        Not Sure: If you don't know the answer: Say "Jai Shri Krishna! I'm not sure, would you like me to look it up?"

    

Context:

{context}

Question:

{question}

"""

In [85]:
new_prompt = ChatPromptTemplate([
    prompt_template
])

In [86]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | new_prompt
    | model
    | StrOutputParser()
)

In [ ]:
st.title("Bhagavad Gita Chatbot")
st.write(
    "This is a chatbot that can answer questions based on the teachings of the Bhagavad Gita."
)
st.write("Please enter your question below and click 'Ask' to get an answer.")
st.write(
    "You can ask questions about the teachings, philosophy, and concepts of the Bhagavad Gita."
)
st.write(
    "For example, you can ask about the meaning of life, the purpose of human existence, the nature of the soul, etc."
)
st.write(
    "Please note that this chatbot is designed to provide information based on the text of the Bhagavad Gita and may not be able to answer all questions."
)
# taking input from user
question = st.text_input("Enter your question here:")
if st.button("Ask"):
    answer = rag_chain.invoke(question)
    st.write(answer)

In [88]:
rag_chain.invoke("Should i do bad to a person get good things")

'Jai Shri Krishna! The Bhagavad Gita emphasizes the importance of righteous conduct and discourages any action that causes harm to others.\n\nHere\'s what the Gita teaches:\n\n* **Karma Yoga:**  The Gita promotes "Karma Yoga," which is selfless action performed without attachment to the results. It encourages doing one\'s duty without expecting anything in return, be it good or bad.\n* **Detachment from Results:** The Gita stresses the importance of detachment from the fruits of our actions. We should not act with the intention of harming others or expecting them to suffer.  \n\nThe passage from Chapter 3, verse 9, states:\n\n>  *“sarva-dharman parityajya mam ekam sharanam vraja ahham tvam sarva-papebhyo mokshayishyami ma shuchah*”\n\n> *“Abandon all varieties of religion and just surrender unto Me. I shall deliver you from all sinful reactions. Do not fear.”*\n\n\nThis verse shows that by surrendering to Krishna and acting with selfless devotion, one can be freed from the cycle of kar

In [89]:
def loading_models():
    return ChatGroq(model="gemma2-9b-it"), OllamaEmbeddings(model="gemma2:9b")

In [92]:
model, embeddings = loading_models()[0], loading_models()[1]

In [94]:
embeddings

OllamaEmbeddings(model='gemma2:9b', base_url=None, client_kwargs={}, _client=<ollama._client.Client object at 0x7f742c32cfd0>, _async_client=<ollama._client.AsyncClient object at 0x7f742c32cfa0>)